In [1]:
# Two lines that remove tensorflow GPU logs
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.models import Sequential, model_from_json
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Activation
from keras.preprocessing.image import ImageDataGenerator
from sklearn import model_selection
from math import ceil

# Loads csv files and appends pixels to X and labels to y
def preprocess_data():
    data = pd.read_csv('data/fer2013.csv')
    labels = pd.read_csv('data/fer2013new.csv') # referente a caracteristica da imagem

    orig_class_names = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt',
                        'unknown', 'NF']

    n_samples = len(data)
    w = 48
    h = 48

    y = np.array(labels[orig_class_names])
    X = np.zeros((n_samples, w, h, 1))
    for i in range(n_samples):
        X[i] = np.fromstring(data['pixels'][i], dtype=int, sep=' ').reshape((h, w, 1))

    return X, y


def clean_data_and_normalize(X, y):
    orig_class_names = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt',
                        'unknown', 'NF']

    # Using mask to remove unknown or NF images
    y_mask = y.argmax(axis=-1)
    mask = y_mask < orig_class_names.index('unknown')
    X = X[mask]
    y = y[mask]

    # Convert to probabilities between 0 and 1
    y = y[:, :-2] * 0.1

    # Add contempt to neutral and remove it
    y[:, 0] += y[:, 7]
    y = y[:, :7]

    # Normalize image vectors
    X = X / 255.0

    return X, y


def split_data(X, y):
    test_size = ceil(len(X) * 0.1)

    # Dividindo o dataset em Train e Test
    x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=test_size)
    x_train, x_val, y_train, y_val = model_selection.train_test_split(x_train, y_train, test_size=test_size)

    return x_train, y_train, x_val, y_val, x_test, y_test


def data_augmentation(x_train): # Data augmentation para balancear o dataset
    shift = 0.1
    datagen = ImageDataGenerator(
        rotation_range=20,
        horizontal_flip=True,
        height_shift_range=shift,
        width_shift_range=shift)
    datagen.fit(x_train)
    return datagen


def show_augmented_images(datagen, x_train, y_train):
    it = datagen.flow(x_train, y_train, batch_size=1)
    plt.figure(figsize=(10, 7))
    for i in range(25):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(it.next()[0][0], cmap='gray')
        # plt.xlabel(class_names[y_train[i]])
    plt.show()


def define_model(input_shape=(48, 48, 1), classes=7):
    num_features = 64

    model = Sequential() # Camada de entrada

    #i wanna build a model with yolo architecture
    model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(2 * num_features, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(2 * num_features, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(2 * 2 * num_features, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(2 * 2 * num_features, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())

    model.add(Dense(2 * 2 * 2 * num_features, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(2 * 2 * num_features, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(2 * num_features, activation='relu'))
    model.add(Dropout(0.5))


    model.add(Dense(classes, activation='softmax'))

    # return model

    # # i wanna build a model with SSD architecture
    # # 1st block
    # model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    # model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # model.add(Dropout(0.5))

    # # 2nd block
    # model.add(Conv2D(2 * num_features, kernel_size=(3, 3), activation='relu'))
    # model.add(Conv2D(2 * num_features, kernel_size=(3, 3), activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # model.add(Dropout(0.5))
    
    # # 3rd block
    # model.add(Conv2D(2 * 2 * num_features, kernel_size=(3, 3), activation='relu'))
    # model.add(Conv2D(2 * 2 * num_features, kernel_size=(3, 3), activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # model.add(Dropout(0.5))


    # # 1st stage 
    # model.add(Conv2D(num_features, kernel_size=(3, 3), input_shape=input_shape))
    # model.add(BatchNormalization())
    # model.add(Activation(activation='relu'))
    # model.add(Conv2D(num_features, kernel_size=(3, 3)))
    # model.add(BatchNormalization())
    # model.add(Activation(activation='relu'))
    # model.add(Dropout(0.5)) #Dropout is a simple and powerful regularization technique for neural networks and deep learning models.

    # # 2nd stage
    # model.add(Conv2D(num_features, (3, 3), activation='relu'))
    # model.add(Conv2D(num_features, (3, 3), activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # # 3rd stage
    # model.add(Conv2D(2 * num_features, kernel_size=(3, 3)))
    # model.add(BatchNormalization())
    # model.add(Activation(activation='relu'))
    # model.add(Conv2D(2 * num_features, kernel_size=(3, 3)))
    # model.add(BatchNormalization())
    # model.add(Activation(activation='relu'))

    # # 4th stage
    # model.add(Conv2D(2 * num_features, (3, 3), activation='relu'))
    # model.add(Conv2D(2 * num_features, (3, 3), activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # # 5th stage
    # model.add(Conv2D(4 * num_features, kernel_size=(3, 3)))
    # model.add(BatchNormalization())
    # model.add(Activation(activation='relu'))
    # model.add(Conv2D(4 * num_features, kernel_size=(3, 3)))
    # model.add(BatchNormalization())
    # model.add(Activation(activation='relu'))

    # model.add(Flatten()) # achatando o modelo

    # # Fully connected neural networks
    # model.add(Dense(1024, activation='relu'))
    # model.add(Dropout(0.2))
    # model.add(Dense(1024, activation='relu'))
    # model.add(Dropout(0.2))

    # model.add(Dense(classes, activation='softmax'))

    return model



def run_model():
    fer_classes = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear']

    X, y = preprocess_data()
    X, y = clean_data_and_normalize(X, y)
    x_train, y_train, x_val, y_val, x_test, y_test = split_data(X, y)
    datagen = data_augmentation(x_train)

    epochs = 200
    batch_size = 64

    print("X_train shape: " + str(x_train.shape))
    print("Y_train shape: " + str(y_train.shape))
    print("X_test shape: " + str(x_test.shape))
    print("Y_test shape: " + str(y_test.shape))
    print("X_val shape: " + str(x_val.shape))
    print("Y_val shape: " + str(y_val.shape))

    # Training model from scratch
    model = define_model(input_shape=x_train[0].shape, classes=len(fer_classes))
    model.summary()
    model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                        steps_per_epoch=len(x_train) // batch_size,
                        validation_data=(x_val, y_val), verbose=2)
    test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=batch_size)

    plot_acc_loss(history)
    save_model_and_weights(model, test_acc)

run_model()

KeyboardInterrupt: 

: 

In [1]:
import visualkeras
visualkeras.layered_view(model, legend=True).show()

NameError: name 'model' is not defined